In [144]:
import re
import spacy
import gensim
import logging
import pandas as pd
import numpy as np
# remove warnings
logging.basicConfig(level=logging.ERROR)
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
documents = pd.read_csv('./datos/simpsons_dataset.csv').dropna().drop_duplicates()
documents = documents.reset_index(drop=True)
print(documents.shape)
display(documents.head())

(126646, 2)


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
main_characters =  ["Lisa Simpson", "Bart Simpson", "Homer Simpson", "Marge Simpson"]
documents = documents[documents["raw_character_text"].isin(main_characters)]
print(documents.shape)
display(documents.head())

(60610, 2)


,raw_character_text,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
7,Bart Simpson,Victory party under the slide!
8,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
10,Lisa Simpson,Do you know where I could find him?


In [4]:
def get_sublist(list, n):
    return [list[i:i+n] for i in range(0, len(list), n)]


In [41]:
def get_vectors_characters(characters: pd.DataFrame) -> pd.DataFrame:
    sentences_big = get_sublist(characters['spoken_words'].tolist(),5)
    df = pd.DataFrame({ 'spoken_words': sentences_big})
    for index,dialogue in df['spoken_words'].items():
        df.loc[index,"spoken_words_concatenated"] = ' '.join(dialogue)
    return df

In [104]:
documents_dialogue_concatenated = documents.groupby("raw_character_text")  \
                                    .apply(lambda x: get_vectors_characters(x)) \
                                    .reset_index("raw_character_text")

In [105]:
def convert_label(label):
    if label == "bart simpson":
        return 0
    elif label == "lisa simpson":
        return 1
    elif label == "homer simpson":
        return 2
    else : 
        return 3

In [106]:
documents_dialogue_concatenated["raw_character_text"] = documents_dialogue_concatenated["raw_character_text"].apply(lambda x: x.lower())
documents_dialogue_concatenated["raw_character_text"] = documents_dialogue_concatenated["raw_character_text"].apply(convert_label)
documents_dialogue_concatenated["spoken_words_concatenated"] = documents_dialogue_concatenated["spoken_words_concatenated"].apply(lambda x: x.lower())
documents_dialogue_concatenated.head()

,raw_character_text,spoken_words,spoken_words_concatenated
0,0,"[Victory party under the slide!, Hey, thanks f...","victory party under the slide! hey, thanks for..."
1,0,"[Somebody must have voted., Uh oh., I demand a...",somebody must have voted. uh oh. i demand a re...
2,0,"[Ah, Dad, if just me, Milhouse and Lewis had v...","ah, dad, if just me, milhouse and lewis had vo..."
3,0,"[Please Dad., What?, Yes sir., They're fightin...",please dad. what? yes sir. they're fighting in...
4,0,"[Dad, I have as much respect for you as I ever...","dad, i have as much respect for you as i ever ..."


In [160]:

sentences = documents_dialogue_concatenated['spoken_words_concatenated'].values
y = documents_dialogue_concatenated['raw_character_text'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.2, random_state=1)

y_train_categorical = tf.one_hot(y_train,4)
y_test_categorical = tf.one_hot(y_test,4)

# Once we have our handles, we format the datasets in a Keras-fit compatible
# format: a tuple of the form (text_data, label).
train_dataset = tf.data.Dataset.from_tensor_slices((sentences_train,y_train_categorical))
test_dataset = tf.data.Dataset.from_tensor_slices((sentences_test,y_test_categorical))

# We also create a dataset with only the textual data in it. This will be used
# to build our vocabulary later on.
text_dataset = tf.data.Dataset.from_tensor_slices((sentences_train))

In [149]:

print('Number of rows in the total set: {}'.format(sentences.shape[0]))
print('Number of rows of bart class: {}'.format(y[y == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y[y == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y[y == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y[y == 3].shape[0]))
print('---------------------------------------------')

print('Number of rows in the training set: {}'.format(sentences_train.shape[0]))
print('Number of rows of bart class: {}'.format(y_train[y_train == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y_train[y_train == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y_train[y_train == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y_train[y_train == 3].shape[0]))
print('---------------------------------------------')

print('Number of rows in the test set: {}'.format(sentences_test.shape[0]))

print('Number of rows of bart class: {}'.format(y_test[y_test == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y_test[y_test == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y_test[y_test == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y_test[y_test == 3].shape[0]))


Number of rows in the total set: 12123
Number of rows of bart class: 2414
Number of rows of lisa class: 2029
Number of rows of homer class: 5195
Number of rows of marge class: 2485
---------------------------------------------
Number of rows in the training set: 9698
Number of rows of bart class: 1913
Number of rows of lisa class: 1598
Number of rows of homer class: 4171
Number of rows of marge class: 2016
---------------------------------------------
Number of rows in the test set: 2425
Number of rows of bart class: 501
Number of rows of lisa class: 431
Number of rows of homer class: 1024
Number of rows of marge class: 469


## Vectorize Layer Binary

In [172]:
vectorize_layer_binary = TextVectorization(
    ngrams=None, 
    max_tokens=100, vocabulary=None,
    output_mode='binary', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_binary.adapt(text_dataset)


In [177]:
modelsequ1 = Sequential()
modelsequ1.add(tf.keras.Input(shape=(1,), dtype=tf.string))
modelsequ1.add(vectorize_layer_binary)
modelsequ1.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ1.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ1.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 
modelsequ1.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_12 (Text  (None, 100)              0         
 Vectorization)                                                  
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                 204       
                                                                 
Total params: 5,254
Trainable params: 5,254
Non-trainable params: 0
_________________________________________________________________


In [175]:
batch_size = 100
epochs = 20

# Fit the model using the train and test datasets.
modelsequ1.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/20
97/97 [==============================] - 3s 20ms/step - loss: 1.3671 - acc: 0.3418 - mse: 0.1845 - val_loss: 1.3446 - val_acc: 0.3975 - val_mse: 0.1808
Epoch 2/20
97/97 [==============================] - 1s 13ms/step - loss: 1.3177 - acc: 0.4147 - mse: 0.1778 - val_loss: 1.3261 - val_acc: 0.4186 - val_mse: 0.1782
Epoch 3/20
97/97 [==============================] - 1s 11ms/step - loss: 1.2991 - acc: 0.4244 - mse: 0.1753 - val_loss: 1.3104 - val_acc: 0.4252 - val_mse: 0.1762
Epoch 4/20
97/97 [==============================] - 1s 11ms/step - loss: 1.2829 - acc: 0.4306 - mse: 0.1732 - val_loss: 1.2963 - val_acc: 0.4301 - val_mse: 0.1743
Epoch 5/20
97/97 [==============================] - 1s 14ms/step - loss: 1.2677 - acc: 0.4353 - mse: 0.1711 - val_loss: 1.2830 - val_acc: 0.4375 - val_mse: 0.1725
Epoch 6/20
97/97 [==============================] - 1s 12ms/step - loss: 1.2531 - acc: 0.4440 - mse: 0.1691 - val_loss: 1.2700 - val_acc: 0.4429 - val_mse: 0.1708
Epoch 7/20
97/97 [====

## Vectorize Layer count

In [184]:
vectorize_layer_count = TextVectorization(
    ngrams=None, 
    max_tokens=100, vocabulary=None,
    output_mode='count', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_count.adapt(text_dataset)

In [185]:
modelsequ2 = Sequential()
modelsequ2.add(tf.keras.Input(shape=(1,), dtype=tf.string))
modelsequ2.add(vectorize_layer_count)
modelsequ2.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ2.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 
modelsequ2.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_15 (Text  (None, 100)              0         
 Vectorization)                                                  
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                 204       
                                                                 
Total params: 5,254
Trainable params: 5,254
Non-trainable params: 0
_________________________________________________________________


In [186]:
# Fit the model using the train and test datasets.
modelsequ2.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/20
97/97 [==============================] - 3s 15ms/step - loss: 1.3935 - acc: 0.3822 - mse: 0.1849 - val_loss: 1.3477 - val_acc: 0.4004 - val_mse: 0.1806
Epoch 2/20
97/97 [==============================] - 1s 13ms/step - loss: 1.3263 - acc: 0.4150 - mse: 0.1775 - val_loss: 1.3143 - val_acc: 0.4165 - val_mse: 0.1766
Epoch 3/20
97/97 [==============================] - 1s 11ms/step - loss: 1.2947 - acc: 0.4297 - mse: 0.1739 - val_loss: 1.2899 - val_acc: 0.4214 - val_mse: 0.1736
Epoch 4/20
97/97 [==============================] - 1s 11ms/step - loss: 1.2699 - acc: 0.4384 - mse: 0.1708 - val_loss: 1.2696 - val_acc: 0.4359 - val_mse: 0.1710
Epoch 5/20
97/97 [==============================] - 1s 11ms/step - loss: 1.2483 - acc: 0.4494 - mse: 0.1680 - val_loss: 1.2502 - val_acc: 0.4445 - val_mse: 0.1684
Epoch 6/20
97/97 [==============================] - 1s 11ms/step - loss: 1.2293 - acc: 0.4602 - mse: 0.1655 - val_loss: 1.2331 - val_acc: 0.4540 - val_mse: 0.1660
Epoch 7/20
97/97 [====

## Vectorize layer tf-idf

In [189]:
vectorize_layer_tfidf = TextVectorization(
    ngrams=None, 
    max_tokens=100, vocabulary=None,
    output_mode='tf-idf', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_tfidf.adapt(text_dataset)

In [190]:
modelsequ3 = Sequential()
modelsequ3.add(tf.keras.Input(shape=(1,), dtype=tf.string))
modelsequ3.add(vectorize_layer_tfidf)
modelsequ3.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ3.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 
modelsequ3.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_17 (Text  (None, 100)              1         
 Vectorization)                                                  
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                 204       
                                                                 
Total params: 5,255
Trainable params: 5,254
Non-trainable params: 1
_________________________________________________________________


In [191]:
# Fit the model using the train and test datasets.
modelsequ2.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/20
97/97 [==============================] - 1s 13ms/step - loss: 1.0495 - acc: 0.5648 - mse: 0.1418 - val_loss: 1.0655 - val_acc: 0.5538 - val_mse: 0.1434
Epoch 2/20
97/97 [==============================] - 1s 10ms/step - loss: 1.0441 - acc: 0.5676 - mse: 0.1411 - val_loss: 1.0612 - val_acc: 0.5538 - val_mse: 0.1428
Epoch 3/20
97/97 [==============================] - 1s 10ms/step - loss: 1.0394 - acc: 0.5691 - mse: 0.1405 - val_loss: 1.0574 - val_acc: 0.5559 - val_mse: 0.1423
Epoch 4/20
97/97 [==============================] - 1s 11ms/step - loss: 1.0347 - acc: 0.5705 - mse: 0.1399 - val_loss: 1.0543 - val_acc: 0.5575 - val_mse: 0.1420
Epoch 5/20
97/97 [==============================] - 1s 12ms/step - loss: 1.0303 - acc: 0.5724 - mse: 0.1393 - val_loss: 1.0513 - val_acc: 0.5575 - val_mse: 0.1416
Epoch 6/20
97/97 [==============================] - 1s 12ms/step - loss: 1.0263 - acc: 0.5760 - mse: 0.1388 - val_loss: 1.0487 - val_acc: 0.5579 - val_mse: 0.1413
Epoch 7/20
97/97 [====